<a href="https://colab.research.google.com/github/fecsaba/automatic_photo/blob/main/auto_get_meta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Drive**

In [1]:
!pip install transformers keybert torch
import os

# Csatoljuk a Google Drive-ot
from google.colab import drive
drive.mount('/content/drive')
!pip install pypng
!pip install Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

# ***Action***

```
'/content/drive/MyDrive/Images'
```



## Meta kiszedése

In [2]:
from PIL import PngImagePlugin, Image
import os
import json

class PngValidator:
    """PNG fájlok érvényességének ellenőrzésére szolgáló osztály.

    Ez az osztály felelős a PNG fájlok formátumának validálásáért. Ellenőrzi, hogy egy fájl
    valóban érvényes PNG formátumú-e, függetlenül a fájl kiterjesztésétől.

    Methods:
        is_valid_png(file_path): Ellenőrzi egy fájl PNG formátumát.
    """

    @staticmethod
    def is_valid_png(file_path):
        """Ellenőrzi, hogy a fájl valóban PNG formátumú-e.

        A metódus megnyitja a fájlt és ellenőrzi annak belső formátumát,
        nem csak a kiterjesztést vizsgálja.

        Args:
            file_path (str): A vizsgálandó fájl teljes elérési útja.

        Returns:
            bool: True ha a fájl valódi PNG formátumú, False egyébként.

        Example:
            >>> validator = PngValidator()
            >>> is_valid = validator.is_valid_png("kep.png")
            >>> print(is_valid)
            True
        """
        try:
            with Image.open(file_path) as img:
                return isinstance(img, PngImagePlugin.PngImageFile) and img.format == 'PNG'
        except Exception:
            return False

class PngFileHandler:
    """PNG fájlok és mappák kezelésére szolgáló osztály.

    Ez az osztály felelős a fájlrendszer műveletek végrehajtásáért,
    beleértve a mappák beolvasását és a PNG fájlok azonosítását.

    Attributes:
        directory_path (str): A feldolgozandó mappa elérési útja.

    Methods:
        set_directory_path(directory_path): Beállítja a feldolgozandó mappa útvonalát.
        get_png_files(): Összegyűjti az érvényes PNG fájlokat a megadott mappából.
    """

    def __init__(self, directory_path=None):
        """Inicializálja a PngFileHandler osztályt.

        Args:
            directory_path (str, optional): A feldolgozandó mappa elérési útja.
                                          Alapértelmezett érték: None.
        """
        self.directory_path = directory_path

    def set_directory_path(self, directory_path):
        """Beállítja a feldolgozandó mappa útvonalát.

        Args:
            directory_path (str): A feldolgozandó mappa elérési útja.
        """
        self.directory_path = directory_path

    def get_png_files(self):
        """Összegyűjti az érvényes PNG fájlokat a megadott mappából.

        A metódus ellenőrzi a megadott mappában található összes fájlt,
        és csak azokat a fájlokat adja vissza, amelyek valóban PNG formátumúak.

        Returns:
            list: Érvényes PNG fájlok teljes elérési útjainak listája.

        Note:
            - Ha nincs megadva mappa útvonal, üres listával tér vissza
            - Ha a mappa nem létezik, üres listával tér vissza
            - Figyelmeztetést ad ki, ha talál olyan fájlt, ami .png kiterjesztésű,
              de nem valódi PNG formátumú
        """
        if not self.directory_path:
            print("Nincs megadva mappa útvonal!")
            return []

        if not os.path.exists(self.directory_path):
            print(f"A megadott mappa nem létezik: {self.directory_path}")
            return []

        png_files = []
        for filename in os.listdir(self.directory_path):
            file_path = os.path.join(self.directory_path, filename)
            if PngValidator.is_valid_png(file_path):
                png_files.append(file_path)
            elif filename.lower().endswith('.png'):
                print(f"\nFigyelmeztetés: A fájl '.png' kiterjesztésű, de nem valódi PNG: {filename}")

        return png_files

class MetadataReader:
    """PNG fájlok metaadatainak olvasására szolgáló osztály.

    Ez az osztály felelős a PNG fájlok metaadatainak kinyeréséért és
    megjelenítéséért. A metaadatok között szerepelhetnek például a készítő,
    a létrehozás dátuma, és egyéb egyedi információk.

    Methods:
        read_metadata(file_path): Beolvassa és megjeleníti egy PNG fájl metaadatait.
    """

    @staticmethod
    def read_metadata(file_path):
        """Beolvassa egy PNG fájl metaadatait.

        Args:
            file_path (str): A PNG fájl teljes elérési útja.

        Returns:
            dict: A metaadatok szótár formátumban, ahol a kulcsok a metaadat mezők nevei,
                 az értékek pedig a hozzájuk tartozó értékek.
                 Hiba esetén None-nal tér vissza.
                 Ha nincsenek metaadatok, üres szótárral tér vissza.

        Note:
            A metódus a következő eseteket kezeli:
            - Sikeres beolvasás esetén kiírja a metaadatokat
            - Ha nincsenek metaadatok, ezt jelzi
            - Ha a fájl nem található, hibaüzenetet ad
            - Egyéb hibák esetén hibaüzenetet ad
        """
        try:
            with Image.open(file_path) as img:
                metadata = img.text
                if metadata:
                    print(f"\nMetaadatok a következő fájlhoz: {os.path.basename(file_path)}")
                    print("-" * 50)
                    for k, v in metadata.items():
                        print(f"**{k}**:\n{v}\n")
                    return metadata
                else:
                    print(f"\nNem találtunk metaadatokat a következő fájlban: {os.path.basename(file_path)}")
                    return {}

        except FileNotFoundError:
            print(f"A fájl nem található: {file_path}")
            return None
        except Exception as e:
            print(f"Hiba történt a következő fájlnál {os.path.basename(file_path)}: {str(e)}")
            return None

class PngMetadataProcessor:
    """PNG fájlok metaadatainak feldolgozására szolgáló fő osztály.

    Ez az osztály koordinálja a teljes feldolgozási folyamatot, összekapcsolva
    a fájlkezelést és a metaadat-olvasást. Egy megadott mappában található összes
    PNG fájlt feldolgozza és összegyűjti azok metaadatait.

    Attributes:
        file_handler (PngFileHandler): A fájlkezelésért felelős objektum.

    Methods:
        process_directory(): Feldolgozza a megadott mappában található összes PNG fájlt.
        save_results_to_json(results): Elmenti az eredményeket JSON fájlba.
    """

    def __init__(self, directory_path=None):
        """Inicializálja a PngMetadataProcessor osztályt.

        Args:
            directory_path (str, optional): A feldolgozandó mappa elérési útja.
                                          Alapértelmezett érték: None.
        """
        self.file_handler = PngFileHandler(directory_path)

    def process_directory(self):
        """Feldolgozza a megadott mappában található összes PNG fájlt.

        A metódus végigmegy a megadott mappában található összes PNG fájlon,
        és összegyűjti azok metaadatait. A folyamat során:
        1. Azonosítja az érvényes PNG fájlokat
        2. Beolvassa minden fájl metaadatait
        3. Összesíti az eredményeket

        Returns:
            dict: Egy szótár, ahol a kulcsok a fájlnevek, az értékek pedig
                 a hozzájuk tartozó metaadatok szótár formátumban.
        """
        print(f"\nPNG fájlok keresése a következő mappában: {self.file_handler.directory_path}")
        print("=" * 70)

        png_files = self.file_handler.get_png_files()
        results = {}

        for file_path in png_files:
            metadata = MetadataReader.read_metadata(file_path)
            if metadata:
                results[os.path.basename(file_path)] = metadata

        if not results:
            print("\nNem találtunk érvényes PNG fájlokat a megadott mappában.")

        return results

    def save_results_to_json(self, results):
        """Elmenti az eredményeket JSON fájlba.

        Args:
            results (dict): A mentendő metaadatok szótár formátumban.

        Note:
            A fájl neve 'png_metadata_results.json' lesz, és az Image mappába kerül mentésre.
            Ha a fájl már létezik, felülírja azt.
        """
        if not results:
            print("\nNincs menthető adat.")
            return

        output_path = os.path.join(self.file_handler.directory_path, "png_metadata_results.json")
        try:
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(results, f, indent=2, ensure_ascii=False)
            print(f"\nAz eredmények sikeresen mentve: {output_path}")
        except Exception as e:
            print(f"\nHiba történt a fájl mentése közben: {str(e)}")

# Példa használat és tesztelés
if __name__ == "__main__":
    # Példa a program használatára
    processor = PngMetadataProcessor("/content/drive/MyDrive/Image/")
    all_metadata = processor.process_directory()

    # Eredmények mentése JSON fájlba
    processor.save_results_to_json(all_metadata)

    # Összesítés
    if all_metadata:
        print("\nÖsszesítés:")
        print("=" * 70)
        print(f"Összesen {len(all_metadata)} érvényes PNG fájl metaadatait olvastuk be.")


PNG fájlok keresése a következő mappában: /content/drive/MyDrive/Image/

Metaadatok a következő fájlhoz: Pinterest 2025-03-04-101909-GGUF-Flux-deis-beta-seed-797091519311320-0.png
--------------------------------------------------
**parameters**:
Clip L  , A cute tabby kitten with a mix of light brown, orange, and white fur, adorned with distinct tabby stripes across its body. The kitten features large, striking blue eyes that are wide open, exuding an alert and curious expression. Its mouth is slightly ajar, revealing a small pink tongue and tiny white teeth, as if it’s meowing or playfully panting. The kitten’s ears stand perked up, with the left ear slightly bent, enhancing its charming look. It peeks through a torn section of cardboard or paper, the edges of which are irregular, rough, and frayed, displaying shades of brown and beige for a realistic texture. The background visible through the tear is a deep dark blue or black, creating a stark contrast with the kitten’s light fur.

## Pozitív prompt kiszedése

### Leírás kiválasztása

In [4]:
import json
import os

# Input JSON fájl elérési útja
input_file_path = "/content/drive/MyDrive/Image/png_metadata_results.json"

# Output JSON fájl elérési útja
output_file_path = "/content/drive/MyDrive/Image/prompts.json"

# Ellenőrizzük, hogy létezik-e az input fájl
if not os.path.exists(input_file_path):
    print(f"Hiba: A(z) {input_file_path} fájl nem található.")
else:
    # Olvassuk be az input JSON-t
    with open(input_file_path, "r") as file:
        data = json.load(file)

    # Eredmények tárolására szolgáló szótár
    prompts_data = {}

    # Végigmegyünk az összes elemen a JSON-ban
    for image_name, details in data.items():
        parameters = details.get("parameters", "")

        # Kinyerjük a 'Clip L' részt
        clip_l_start = parameters.find("Clip L")
        if clip_l_start != -1:
            t5xxl_start = parameters.find("T5XXL", clip_l_start)
            negative_prompt_start = parameters.find("\nNegative prompt:", clip_l_start)

            # Ha a 'T5XXL' vagy '\nNegative prompt:' nem található, kezeljük az esetet
            if t5xxl_start == -1 or negative_prompt_start == -1:
                continue

            clip_l_text = parameters[clip_l_start + len("Clip L"):t5xxl_start].strip().rstrip(',')

        # Kinyerjük a 'T5XXL' részt
        t5xxl_start = parameters.find("T5XXL")
        if t5xxl_start != -1:
            negative_prompt_start = parameters.find("\nNegative prompt:", t5xxl_start)
            if negative_prompt_start == -1:
                continue

            t5xxl_text = parameters[t5xxl_start + len("T5XXL"):negative_prompt_start].strip().rstrip(',')

        # Hozzáadjuk az eredményekhez
        prompts_data[image_name] = {
            "clip_l": clip_l_text,
            "t5xxl": t5xxl_text
        }

    # Mentjük az eredményeket egy új JSON fájlba
    with open(output_file_path, "w") as output_file:
        json.dump(prompts_data, output_file, indent=4)

    print(f"Sikeresen létrehoztuk a(z) {output_file_path} fájlt!")

Sikeresen létrehoztuk a(z) /content/drive/MyDrive/Image/prompts.json fájlt!


### Szöveg generálás

### Tisztítás

In [5]:
import json

# Input JSON fájl elérési útja
input_file_path = "/content/drive/MyDrive/Image/prompts.json"

# Output JSON fájl elérési útja
output_file_path = "/content/drive/MyDrive/Image/cleaned_prompts.json"

# Ellenőrizzük, hogy létezik-e az input fájl
if not os.path.exists(input_file_path):
    print(f"Hiba: A(z) {input_file_path} fájl nem található.")
else:
    # Olvassuk be az input JSON-t
    with open(input_file_path, "r") as file:
        data = json.load(file)

    # Eredmények tárolására szolgáló szótár
    cleaned_data = {}

    # Végigmegyünk az összes képen
    for image_name, details in data.items():
        clip_l_text = details.get("clip_l", "").strip().lstrip(", ").rstrip(", ")
        t5xxl_text = details.get("t5xxl", "").strip().lstrip(", ").rstrip(", ")

        # Hozzáadjuk az átalakított adatokat az új szótárhoz
        cleaned_data[image_name] = {
            "clip_l": clip_l_text,
            "t5xxl": t5xxl_text
        }

    # Mentjük az eredményeket egy új JSON fájlba
    with open(output_file_path, "w") as output_file:
        json.dump(cleaned_data, output_file, indent=4)

    print(f"Sikeresen létrehoztuk a(z) {output_file_path} fájlt!")

Sikeresen létrehoztuk a(z) /content/drive/MyDrive/Image/cleaned_prompts.json fájlt!


#### Képolvasás

In [6]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
Képleírás generáló szkript
Ez a szkript a Google Colab környezetben fut, és a következőket végzi:
- Képek bőséges leírásának generálása BLIP modellel (~500 karakter).
- Az eredmények mentése a /content/drive/MyDrive/Image/cleaned_prompts.json fájlba img_desc mezővel.
- Az inputok a képek, a prompt csak tárolásra szolgál, és csak az img_desc mező kerül hozzáadásra a meglévőkhöz.

A finomhangolható paraméterek a BLIP_CONFIG szótárban vannak definiálva a script elején,
hogy könnyen módosíthatóak legyenek a kimenet testreszabásához.
"""

from PIL import Image
import json
import os
from google.colab import drive
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch

# Paraméterek konfigurációja
# BLIP modell paraméterei (képleírás generáláshoz)
BLIP_CONFIG = {
    "max_new_tokens": 320,  # Maximális új tokenek száma a generáláshoz (~500 karakter)
    # Magyarázat: Meghatározza a generálandó új tokenek maximális számát. Egy token ~1.5-2 karakter,
    # így ez a leírás hosszát szabályozza. Növelés (pl. 350) hosszabb, csökkentés (pl. 300) rövidebb leírást eredményez.
    "num_beams": 4,  # Beam search száma a legjobb szekvenciák kiválasztásához
    # Magyarázat: Minél nagyobb, annál jobb koherencia, de lassabb. 3 stabil, 5 részletgazdag, de terhelő lehet.
    "length_penalty": 2.8,  # Hossz büntetési tényező
    # Magyarázat: >1 hosszabb, <1 rövidebb szöveget ösztönöz. 2.5-3.0 hosszabb, 2.0 alatti rövidebb leírást ad.
    "no_repeat_ngram_size": 5,  # Ismétlődő n-gramok száma, amit el kell kerülni
    # Magyarázat: Csökkenti a redundanciát. 2 rugalmas, 5 szigorúbb, 4 kompromisszum.
    "do_sample": False,  # Determinisztikus vs. sztochasztikus generálás
    # Magyarázat: False konzisztens, True kreatívabb, de instabil. True esetén temperature finomhangolható.
    "early_stopping": False  # Korai leállítás a generálásnál
    # Magyarázat: False a max_new_tokens-ig megy, True korán leáll, ha befejezettnek ítéli.
}

class FileManager:
    def __init__(self, project_dir='/content/drive/MyDrive/Image', filename='cleaned_prompts.json'):
        """Inicializálja a fájlkezelőt a megadott könyvtár- és fájlnévvel."""
        self.project_dir = project_dir
        self.filename = os.path.join(project_dir, filename)
        self.data = {}

    def mount_drive(self):
        """Csatlakoztatja a Google Drive-ot a Colab környezetben."""
        print("Mounting Google Drive...")
        drive.mount('/content/drive', force_remount=True)

    def load_json(self):
        """Betölti a meglévő JSON fájlt, vagy létrehoz egy újat, ha nem létezik."""
        if os.path.exists(self.filename):
            try:
                with open(self.filename, 'r', encoding='utf-8') as f:
                    self.data = json.load(f)
            except json.JSONDecodeError:
                print(f"Figyelmeztetés: '{self.filename}' érvénytelen JSON fájl.")
                self.data = {}
        else:
            print(f"'{self.filename}' nem található. Új fájl lesz létrehozva.")
        return self.data

    def save_json(self):
        """Elmenti az adatokat a JSON fájlba."""
        try:
            with open(self.filename, 'w', encoding='utf-8') as f:
                json.dump(self.data, f, ensure_ascii=False, indent=2)
            print(f"'{self.filename}' mentve!")
        except Exception as e:
            print(f"Hiba a fájl mentése közben: {e}")

    def list_images(self):
        """Kilistázza a megadott könyvtárban lévő képeket (.png, .jpg, .jpeg kiterjesztéssel)."""
        if not os.path.exists(self.project_dir):
            print(f"Hiba: '{self.project_dir}' könyvtár nem található.")
            return []
        return [f for f in os.listdir(self.project_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

class ImageDescriber:
    def __init__(self, processor, model):
        """Inicializálja a képelemzőt a BLIP modellel."""
        self.processor = processor
        self.model = model

    def load_image(self, image_path):
        """Betölti a képet RGB formátumba."""
        try:
            return Image.open(image_path).convert("RGB")
        except Exception as e:
            print(f"Hiba a(z) '{image_path}' betöltése közben: {e}")
            return None

    def generate_description(self, image_path):
        """Generál egy ~500 karakteres leírást a képről a BLIP modellel."""
        image = self.load_image(image_path)
        if image is None:
            return "Nem sikerült betölteni a képet."
        inputs = self.processor(image, return_tensors="pt").to('cuda')
        out = self.model.generate(
            **inputs,
            max_new_tokens=BLIP_CONFIG["max_new_tokens"],
            num_beams=BLIP_CONFIG["num_beams"],
            length_penalty=BLIP_CONFIG["length_penalty"],
            no_repeat_ngram_size=BLIP_CONFIG["no_repeat_ngram_size"],
            do_sample=BLIP_CONFIG["do_sample"],
            early_stopping=BLIP_CONFIG["early_stopping"]
        )
        desc = self.processor.decode(out[0], skip_special_tokens=True)
        if len(desc) > 500:
            desc = desc[:500]  # Maximum 500 karakter
        return desc

def main():
    """Fő függvény, amely a képleírás generálási folyamatot vezérli."""
    # GPU elérhetőség ellenőrzése
    if not torch.cuda.is_available():
        print("Figyelmeztetés: GPU nem elérhető. A feldolgozás lassabb lehet.")
    else:
        print("GPU észlelve. GPU-t használjuk a feldolgozáshoz.")

    # Fájlkezelő inicializálása és Drive csatlakoztatása
    file_manager = FileManager()
    file_manager.mount_drive()
    data = file_manager.load_json()
    image_files = file_manager.list_images()

    # Képfeldolgozás ellenőrzése
    if not image_files:
        print("Nincsenek képek a megadott könyvtárban.")
        return

    print(f"Talált képek: {image_files}")

    # Modell inicializálása
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to('cuda')
    describer = ImageDescriber(processor, model)

    # Képfeldolgozás és leírás generálás
    for filename in image_files:
        image_path = os.path.join(file_manager.project_dir, filename)
        print(f"Feldolgozás: {filename}...")

        try:
            # Prompt csak tárolásra, nem használjuk a generáláshoz
            prompt = data.get(filename, {}).get("prompt", None) or \
                     data.get(filename, {}).get("clip_l", None) or \
                     data.get(filename, {}).get("t5xxl", None) or "Nincs megadva prompt"
            image_desc = describer.generate_description(image_path)
            print(f"{filename} leírása: {image_desc} (Hossz: {len(image_desc)})")
            print(f"{filename} Összesen: {len(image_desc)}")

            if filename not in data:
                data[filename] = {}
            # Csak az img_desc mezőt adjuk hozzá, a meglévőket megtartjuk
            if "img_desc" not in data[filename]:
                data[filename]["img_desc"] = image_desc
        except Exception as e:
            print(f"Hiba a(z) '{filename}' feldolgozása közben: {e}")

    # Eredmények mentése a cleaned_prompts.json fájlba
    file_manager.data = data
    file_manager.save_json()

if __name__ == "__main__":
    main()

GPU észlelve. GPU-t használjuk a feldolgozáshoz.
Mounting Google Drive...
Mounted at /content/drive
Talált képek: ['Pinterest 2025-03-04-101909-GGUF-Flux-deis-beta-seed-797091519311320-0.png', 'Pinterest 2025-03-04-000150-GGUF-Flux-deis-beta-seed-797091519311320-0.png', 'Pinterest 2025-03-03-232739-GGUF-Flux-deis-beta-seed-797091519311320-0.png', 'Pinterest 2025-03-03-232030-GGUF-Flux-deis-beta-seed-797091519311320-0.png', 'Pinterest 2025-03-03-235528-GGUF-Flux-deis-beta-seed-797091519311320-0.png', 'Pinterest 2025-03-04-000857-GGUF-Flux-deis-beta-seed-797091519311320-0.png', 'Pinterest 2025-03-04-103748-GGUF-Flux-deis-beta-seed-740320524205577-0.png', 'Pinterest 2025-03-04-104314-GGUF-Flux-deis-beta-seed-849961598060900-0.png', 'Pinterest 2025-03-03-233837-GGUF-Flux-deis-beta-seed-797091519311320-0.png', 'Pinterest 2025-03-04-102540-GGUF-Flux-deis-beta-seed-740320524205577-0.png', 'Pinterest 2025-03-04-101054-GGUF-Flux-deis-beta-seed-797091519311320-0.png']
Feldolgozás: Pinterest 2025

# **Auto input generálás**

In [7]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
SEO és CTA generáló szkript
Ez a szkript a Google Colab környezetben fut, és a következőket végzi:
- A cleaned_prompts.json fájlban lévő clip_l, t5xxl és img_desc mezők alapján kb. 400 karakteres SEO leírást generál.
- SEO címet generál (max 100 karakter), releváns CTA-t (AI-val generálva a 3 mezőből röviden és frappánsan),
  5 kulcskifejezést, feldolgozási és feltöltési időt, valamint egy üres link mezőt.
- Az eredményeket egy új images_data.json fájlba menti a megadott struktúrában.

A finomhangolható paraméterek a BART_CONFIG és KEYBERT_CONFIG szótárakban vannak definiálva a script elején.

Megjegyzés: A BART modell figyelmeztethet, hogy a max_length nagyobb, mint az input_length (pl. 350 vs. 301 token).
Ez normális a hosszabb leírásokhoz, de csökkenthető (pl. 150-re), ha rövidebb összefoglaló kell.
"""

import json
import os
from google.colab import drive
from transformers import pipeline
from keybert import KeyBERT
import torch
from datetime import datetime

# Paraméterek konfigurációja
# BART modell paraméterei (SEO elemek generálásához)
BART_CONFIG = {
    "title_max_length": 20,  # Csökkentett érték rövidebb címekért
    "title_min_length": 5,
    "desc_max_length": 500,  # Növelt érték a hosszabb leírásért
    "desc_min_length": 300,  # Növelt érték a minimális tartalomhoz
    "cta_max_length": 20,  # Növelt érték a kreatívabb CTA-hoz
    "cta_min_length": 5,
    "do_sample": True,
    "temperature": 0.8
}

KEYBERT_CONFIG = {
    "top_n": 20,  # Növelt érték több variációért
    "keyphrase_ngram_range": (1, 2)
}

# KeyBERT paraméterei (kulcsszavak kinyeréséhez)
KEYBERT_CONFIG = {
    "top_n": 20,  # Maximális kulcsszavak száma a szűrés előtt
    # Magyarázat: Mennyi kulcsszót vizsgáljunk, az első 5 a legrelevánsabb. 10-20 között állítható.
    "keyphrase_ngram_range": (1, 2)  # Kulcsszavak n-gram tartománya
    # Magyarázat: 1-2 szavas kulcsszavakat engedélyez. (1, 3)-ra növelve 3 szavasakat is kaphatunk.
}

class FileManager:
    def __init__(self, input_filename='cleaned_prompts.json', output_filename='images_data.json', project_dir='/content/drive/MyDrive/Image'):
        """Inicializálja a fájlkezelőt a bemenet és kimenet fájlnevekkel."""
        self.project_dir = project_dir
        self.input_filename = os.path.join(project_dir, input_filename)
        self.output_filename = os.path.join(project_dir, output_filename)
        self.data = {}

    def mount_drive(self):
        """Csatlakoztatja a Google Drive-ot a Colab környezetben."""
        print("Mounting Google Drive...")
        drive.mount('/content/drive', force_remount=True)

    def load_json(self):
        """Betölti a bemenet JSON fájlt, vagy létrehoz egy üreset, ha nem létezik."""
        if os.path.exists(self.input_filename):
            try:
                with open(self.input_filename, 'r', encoding='utf-8') as f:
                    self.data = json.load(f)
            except json.JSONDecodeError:
                print(f"Figyelmeztetés: '{self.input_filename}' érvénytelen JSON fájl.")
                self.data = {}
        else:
            print(f"'{self.input_filename}' nem található. Üres adatstruktúrával folytatom.")
        return self.data

    def save_json(self):
        """Elmenti az adatokat a kimeneti JSON fájlba."""
        try:
            with open(self.output_filename, 'w', encoding='utf-8') as f:
                json.dump(self.data, f, ensure_ascii=False, indent=2)
            print(f"'{self.output_filename}' mentve!")
        except Exception as e:
            print(f"Hiba a fájl mentése közben: {e}")

class ImageDescriber:
    def __init__(self):
        """Inicializálja a SEO, CTA és kulcsszó generáló eszközöket."""
        self.summary_generator = pipeline("summarization", model="facebook/bart-large-cnn", framework="pt")
        self.kw_model = KeyBERT()

    def generate_seo_description(self, text):
        """Generál egy kb. 400 karakteres SEO leírást a bemenetből."""
        combined_text = f"Summarize for SEO description: {text}"
        seo_desc = self.summary_generator(combined_text,
                                         max_length=BART_CONFIG["desc_max_length"],
                                         min_length=BART_CONFIG["desc_min_length"],
                                         do_sample=BART_CONFIG["do_sample"],
                                         temperature=BART_CONFIG["temperature"])[0]['summary_text']
        if len(seo_desc) > 400:
            seo_desc = seo_desc[:400]
        return seo_desc

    def generate_seo_title(self, text):
        """Generál egy max 100 karakteres SEO címet a bemenetből."""
        combined_text = f"Summarize for SEO title: {text}"
        seo_title = self.summary_generator(combined_text,
                                         max_length=BART_CONFIG["title_max_length"],
                                         min_length=BART_CONFIG["title_min_length"],
                                         do_sample=BART_CONFIG["do_sample"],
                                         temperature=BART_CONFIG["temperature"])[0]['summary_text']
        seo_title = f"{seo_title[:80].strip()} - Pin It!"
        if len(seo_title) > 100:
            seo_title = seo_title[:97] + "..."
        return seo_title

    def generate_cta(self, text):
      """Generál egy röviden és frappánsan releváns CTA-t a bemenetből AI-val, cselekvésre ösztönözve."""
      combined_text = f"Summarize for CTA action: {text}"
      cta = self.summary_generator(combined_text,
                                  max_length=BART_CONFIG["cta_max_length"],
                                  min_length=BART_CONFIG["cta_min_length"],
                                  do_sample=BART_CONFIG["do_sample"],
                                  temperature=BART_CONFIG["temperature"])[0]['summary_text']
      # CTA-t relevánsan tartjuk, formázzuk, cselekvésre ösztönző szavakkal
      cta = cta.strip().capitalize()
      action_words = ["Discover", "See", "Admire", "Explore", "Meet"]
      if not cta or len(cta.split()) < 2:
          cta = f"{action_words[0]} now"
      else:
          cta_words = cta.split()
          if cta_words[0] not in action_words and len(cta_words) > 1:
              cta = f"{action_words[0]} {cta_words[0]}"
          elif cta_words[0] not in action_words:
              cta = f"{action_words[0]} the scene"
      return f"📌 **{cta}** ✨"

    def generate_keywords(self, text):
        """Generál 5 releváns kulcskifejezést a szövegből."""
        keywords = [kw[0] for kw in self.kw_model.extract_keywords(text,
                                                                  keyphrase_ngram_range=KEYBERT_CONFIG["keyphrase_ngram_range"],
                                                                  stop_words='english',
                                                                  top_n=KEYBERT_CONFIG["top_n"])[:5]]
        return keywords

def main():
    """Fő függvény, amely a SEO, CTA és kulcsszó generálási folyamatot vezérli."""
    if not torch.cuda.is_available():
        print("Figyelmeztetés: GPU nem elérhető. A feldolgozás lassabb lehet.")
    else:
        print("GPU észlelve. GPU-t használjuk a feldolgozáshoz.")

    # Fájlkezelő inicializálása és Drive csatlakoztatása
    file_manager = FileManager()
    file_manager.mount_drive()
    input_data = file_manager.load_json()

    # Kimenet inicializálása
    output_data = {}

    # SEO, CTA és kulcsszó generálás minden képhez
    describer = ImageDescriber()
    for filename in input_data.keys():
        print(f"Feldolgozás: {filename}...")
        try:
            # Összevonjuk a clip_l, t5xxl és img_desc mezőket
            combined_text = ""
            if input_data[filename].get("clip_l"):
                combined_text += input_data[filename]["clip_l"] + " "
            if input_data[filename].get("t5xxl"):
                combined_text += input_data[filename]["t5xxl"] + " "
            if input_data[filename].get("img_desc"):
                combined_text += input_data[filename]["img_desc"] + " "

            if combined_text:
                # SEO leírás generálása
                seo_description = describer.generate_seo_description(combined_text)
                # SEO cím generálása
                seo_title = describer.generate_seo_title(combined_text)
                # CTA generálása
                cta = describer.generate_cta(combined_text)
                # Kulcskifejezések generálása
                keywords = describer.generate_keywords(combined_text)
                # Dátumok
                processed_at = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                uploaded_at = ""

                # Kimenet struktúrába rendezése
                output_data[filename] = {
                    "seo_description": seo_description,
                    "seo_title": seo_title,
                    "link": "",
                    "processed_at": processed_at,
                    "uploaded_at": uploaded_at,
                    "keywords": keywords,
                    "cta": cta
                }
            else:
                print(f"Nincs elegendő adat {filename} számára a feldolgozáshoz.")
        except Exception as e:
            print(f"Hiba a(z) '{filename}' feldolgozása közben: {e}")

    # Eredmények mentése az images_data.json fájlba
    file_manager.data = output_data
    file_manager.save_json()

if __name__ == "__main__":
    main()

GPU észlelve. GPU-t használjuk a feldolgozáshoz.
Mounting Google Drive...
Mounted at /content/drive


Device set to use cuda:0


Feldolgozás: Pinterest 2025-03-04-101909-GGUF-Flux-deis-beta-seed-797091519311320-0.png...


Your max_length is set to 500, but your input_length is only 355. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=177)


Feldolgozás: Pinterest 2025-03-04-000150-GGUF-Flux-deis-beta-seed-797091519311320-0.png...


...

Feldolgozás: Pinterest 2025-03-03-233837-GGUF-Flux-deis-beta-seed-797091519311320-0.png...


Your max_length is set to 500, but your input_length is only 496. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=248)


Feldolgozás: Pinterest 2025-03-04-102540-GGUF-Flux-deis-beta-seed-740320524205577-0.png...


Your max_length is set to 500, but your input_length is only 471. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=235)


Feldolgozás: Pinterest 2025-03-04-101054-GGUF-Flux-deis-beta-seed-797091519311320-0.png...
'/content/drive/MyDrive/Image/images_data.json' mentve!


### **SEO generation** Innen nem jó

In [ ]:
# Szükséges könyvtárak importálása
from google.colab import drive, files
import json
import os
from transformers import pipeline

class FileManager:
    """Osztály a Google Drive fájlkezeléséhez és JSON műveletekhez."""

    def __init__(self, project_dir='/content/drive/MyDrive/Image', filename='cleaned_output.json'):
        """Inicializálja a fájlkezelőt a megadott könyvtárral és fájlnévvel."""
        self.project_dir = project_dir
        self.filename = os.path.join(project_dir, filename)
        self.data = None

    def mount_drive(self):
        """Csatolja a Google Drive-ot a Colab környezethez."""
        print("Csatolás a Google Drive-hoz...")
        drive.mount('/content/drive', force_remount=True)

    def load_json(self):
        """Betölti a JSON fájlt, ha létezik, különben hibát jelez."""
        if not os.path.exists(self.filename):
            print(f"Hiba: A '{self.filename}' nem található a projektkönyvtárban!")
            raise FileNotFoundError("Kérlek, futtasd az előző szkriptet először!")
        print(f"A '{self.filename}' megtalálva, betöltés...")
        with open(self.filename, 'r', encoding='utf-8') as f:
            self.data = json.load(f)
        return self.data

    def save_json(self):
        """Visszaírja a frissített adatokat a JSON fájlba."""
        print("Frissített fájl mentése...")
        with open(self.filename, 'w', encoding='utf-8') as f:
            json.dump(self.data, f, ensure_ascii=False, indent=2)

    def download_file(self):
        """Letölti a frissített JSON fájlt a böngészőbe."""
        print(f"A '{self.filename}' frissítve, letöltheted!")
        files.download(self.filename)

class SEOGenerator:
    """Osztály SEO leírások generálására, stílus- és technikai részletek nélkül."""

    def __init__(self, model_name='facebook/bart-large-cnn'):
        """Inicializálja a generátort a megadott modellel."""
        print(f"Betöltés a '{model_name}' modellhez a leírásokhoz...")
        self.summarizer = pipeline("summarization", model=model_name)
        # Stílus- és technikai kulcsszavak kiszűrése
        self.technical_style_terms = {
            "8k", "resolution", "cinematic", "photorealistic", "hyperrealism", "dynamic", "motion", "capture",
            "crisp", "vibrant", "colors", "high", "detail", "textures", "sharp", "focus", "depth", "field",
            "effect", "emphasize", "subject", "warm", "golden", "hour", "tones", "clean", "polished", "finish",
            "inspired", "high-fashion", "photography", "film", "stills", "ultra-detailed", "texture", "lifelike",
            "eyes", "soft", "bokeh", "effects", "lights", "background", "perfection", "style", "DSLR", "photo",
            "Fujifilm", "X-T4", "f", "2.8", "natural", "lighting", "composition", "variety", "conditions",
            "Adobe", "After", "Effects", "software", "realistic", "striking", "visible", "enhance"
        }

    def filter_technical_style_terms(self, prompt):
        """Eltávolítja a stílus- és technikai kulcsszavakat a promptból."""
        words = prompt.lower().split()
        filtered_words = [word for word in words if word not in self.technical_style_terms]
        return " ".join(filtered_words)

    def generate_seo_description(self, prompt):
        """Generál egy maximum 500 karakteres SEO leírást a promptból, stílus- és technikai részletek nélkül."""
        print(f"SEO leírás generálása a promptból (hossz: {len(prompt)} karakter)...")
        # Technikai és stílus részletek kiszűrése
        filtered_prompt = self.filter_technical_style_terms(prompt)
        print(f"Szűrt prompt: '{filtered_prompt}' ({len(filtered_prompt)} karakter)")
        # Rövid összefoglaló generálása, a fő témára fókuszálva
        summary = self.summarizer(filtered_prompt, max_length=80, min_length=20, do_sample=False)[0]['summary_text']
        seo_desc = summary.strip()[:500]
        # Redundancia eltávolítása
        words = seo_desc.split()
        seo_desc = " ".join(sorted(set(words), key=words.index))[:500]
        print(f"Generált leírás: '{seo_desc}' ({len(seo_desc)} karakter)")
        return seo_desc

class SEOTitleGenerator:
    """Osztály SEO címek generálására a promptból, a fő témára fókuszálva."""

    def __init__(self, model_name='facebook/bart-large-cnn'):
        """Inicializálja a címgenerátort a megadott modellel."""
        print(f"Betöltés a '{model_name}' modellhez a címekhez...")
        self.summarizer = pipeline("summarization", model=model_name)

    def generate_seo_title(self, prompt):
        """Generál egy maximum 100 karakteres SEO címet a promptból, a fő témára koncentrálva."""
        print(f"SEO cím generálása a promptból (hossz: {len(prompt)} karakter)...")
        short_prompt = " ".join(prompt.split()[:50])
        input_text = f"Summarize the main subject of: {short_prompt}. Focus on the key subject, keep it short, max 100 characters."
        summary = self.summarizer(input_text, max_length=15, min_length=5, do_sample=False)[0]['summary_text']
        seo_title = summary.strip()[:100]
        if len(seo_title) == 100 and not seo_title.endswith('.'):
            seo_title = seo_title[:-1] + '.'
        print(f"Generált cím: '{seo_title}' ({len(seo_title)} karakter)")
        return seo_title

def main():
    """Fő függvény az osztályok használatához és a folyamat vezérléséhez."""
    file_manager = FileManager()
    file_manager.mount_drive()
    data = file_manager.load_json()

    seo_generator = SEOGenerator()
    print("SEO leírások ellenőrzése és generálása...")
    for filename, entry in data.items():
        if isinstance(entry, dict):
            if "seo_description" not in entry or len(entry["seo_description"]) > 500:
                print(f"Frissítés szükséges: {filename}...")
                entry["seo_description"] = seo_generator.generate_seo_description(entry["prompt"])
        else:
            print(f"Átalakítás szótárrá és leírás generálása: {filename}...")
            prompt = entry
            seo_desc = seo_generator.generate_seo_description(prompt)
            data[filename] = {"prompt": prompt, "seo_description": seo_desc}

    seo_title_generator = SEOTitleGenerator()
    print("SEO címek ellenőrzése és generálása...")
    for filename, entry in data.items():
        print(f"Feldolgozás: {filename}...")
        seo_title = seo_title_generator.generate_seo_title(entry["prompt"])
        entry["seo_title"] = seo_title

    file_manager.data = data
    file_manager.save_json()
    file_manager.download_file()

if __name__ == "__main__":
    main()

### Képfelismerés

In [4]:
from PIL import Image
import json
import os
from google.colab import drive
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
from datetime import datetime

class FileManager:
    def __init__(self, project_dir='/content/drive/MyDrive/Image', filename='cleaned_output.json'):
        self.project_dir = project_dir
        self.filename = os.path.join(project_dir, filename)
        self.data = {}

    def mount_drive(self):
        print("Mounting Google Drive...")
        drive.mount('/content/drive', force_remount=True)

    def load_json(self):
        if os.path.exists(self.filename):
            try:
                with open(self.filename, 'r', encoding='utf-8') as f:
                    self.data = json.load(f)
            except json.JSONDecodeError:
                print(f"Figyelmeztetés: '{self.filename}' érvénytelen JSON fájl.")
                self.data = {}
        else:
            print(f"'{self.filename}' nem található. Új fájl lesz létrehozva.")
        return self.data

    def save_json(self):
        try:
            with open(self.filename, 'w', encoding='utf-8') as f:
                json.dump(self.data, f, ensure_ascii=False, indent=2)
            print(f"'{self.filename}' mentve!")
        except Exception as e:
            print(f"Hiba a fájl mentése közben: {e}")

    def list_images(self):
        if not os.path.exists(self.project_dir):
            print(f"Hiba: '{self.project_dir}' könyvtár nem található.")
            return []
        return [f for f in os.listdir(self.project_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

class ImageDescriber:
    def __init__(self, processor, model):
        self.processor = processor
        self.model = model

    def load_image(self, image_path):
        try:
            return Image.open(image_path).convert("RGB")
        except Exception as e:
            print(f"Hiba a(z) '{image_path}' betöltése közben: {e}")
            return None

    def generate_description(self, image_path):
        image = self.load_image(image_path)
        if image is None:
            return "Nem sikerült betölteni a képet."
        inputs = self.processor(image, return_tensors="pt").to('cuda')
        out = self.model.generate(
            **inputs,
            max_new_tokens=500,  # Célzottan 450-500 karakter körüli leírás
            num_beams=3,         # Több variáció a részletesebb leírásért
            length_penalty=2.8,  # Erősebb ösztönzés hosszabb szövegre
            no_repeat_ngram_size=2,
            do_sample=False,     # Következetes generálás
            early_stopping=False # Ne álljon meg korán
        )
        desc = self.processor.decode(out[0], skip_special_tokens=True)
        return desc

def main():
    if not torch.cuda.is_available():
        print("Figyelmeztetés: GPU nem elérhető. A feldolgozás lassabb lehet.")
    else:
        print("GPU észlelve. GPU-t használjuk a feldolgozáshoz.")

    file_manager = FileManager()
    file_manager.mount_drive()
    data = file_manager.load_json()
    image_files = file_manager.list_images()

    if not image_files:
        print("Nincsenek képek a megadott könyvtárban.")
        return

    print(f"Talált képek: {image_files}")

    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to('cuda')
    describer = ImageDescriber(processor, model)

    for filename in image_files:
        image_path = os.path.join(file_manager.project_dir, filename)
        print(f"Feldolgozás: {filename}...")

        try:
            prompt = data.get(filename, {}).get("prompt", None)  # Csak tárolásra, nem használatra
            image_desc = describer.generate_description(image_path)
            print(f"{filename} leírása: {image_desc} (Hossz: {len(image_desc)})")

            processed_at = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            if filename not in data:
                data[filename] = {}
            data[filename].update({
                "prompt": prompt or "Nincs megadva prompt",
                "image_description": image_desc,
                "link": "",
                "processed_at": processed_at,
                "uploaded_at": ""
            })
        except Exception as e:
            print(f"Hiba a(z) '{filename}' feldolgozása közben: {e}")

    file_manager.data = data
    file_manager.save_json()

if __name__ == "__main__":
    main()

GPU észlelve. GPU-t használjuk a feldolgozáshoz.
Mounting Google Drive...
Mounted at /content/drive
'/content/drive/MyDrive/Image/cleaned_output.json' nem található. Új fájl lesz létrehozva.
Talált képek: ['Pinterest 2025-03-04-101909-GGUF-Flux-deis-beta-seed-797091519311320-0.png', 'Pinterest 2025-03-04-000150-GGUF-Flux-deis-beta-seed-797091519311320-0.png', 'Pinterest 2025-03-03-232739-GGUF-Flux-deis-beta-seed-797091519311320-0.png', 'Pinterest 2025-03-03-232030-GGUF-Flux-deis-beta-seed-797091519311320-0.png', 'Pinterest 2025-03-03-235528-GGUF-Flux-deis-beta-seed-797091519311320-0.png', 'Pinterest 2025-03-04-000857-GGUF-Flux-deis-beta-seed-797091519311320-0.png', 'Pinterest 2025-03-04-103748-GGUF-Flux-deis-beta-seed-740320524205577-0.png', 'Pinterest 2025-03-04-104314-GGUF-Flux-deis-beta-seed-849961598060900-0.png', 'Pinterest 2025-03-03-233837-GGUF-Flux-deis-beta-seed-797091519311320-0.png', 'Pinterest 2025-03-04-102540-GGUF-Flux-deis-beta-seed-740320524205577-0.png', 'Pinterest 202

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

Feldolgozás: Pinterest 2025-03-04-101909-GGUF-Flux-deis-beta-seed-797091519311320-0.png...
Pinterest 2025-03-04-101909-GGUF-Flux-deis-beta-seed-797091519311320-0.png leírása: there is a cat that is sitting in a cardboard box with its mouth open and it ' s mouth wide open, looking out of the hole in the box (Hossz: 132)
Feldolgozás: Pinterest 2025-03-04-000150-GGUF-Flux-deis-beta-seed-797091519311320-0.png...
Pinterest 2025-03-04-000150-GGUF-Flux-deis-beta-seed-797091519311320-0.png leírása: a drawing of a girl with a hat and flowers in her hair (Hossz: 54)
Feldolgozás: Pinterest 2025-03-03-232739-GGUF-Flux-deis-beta-seed-797091519311320-0.png...
Pinterest 2025-03-03-232739-GGUF-Flux-deis-beta-seed-797091519311320-0.png leírása: there is a cat that is looking out of a hole in the wall (Hossz: 56)
Feldolgozás: Pinterest 2025-03-03-232030-GGUF-Flux-deis-beta-seed-797091519311320-0.png...
Pinterest 2025-03-03-232030-GGUF-Flux-deis-beta-seed-797091519311320-0.png leírása: there is a cat tha

### SEO cím és leírás kulcsszavakkal

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
Képfeldolgozó és SEO-elemző szkript
Ez a szkript a Google Colab környezetben fut, és a következőket végzi:
- Képek leírásának generálása BLIP modellel (~500 karakter).
- SEO-barát cím, leírás és kulcsszavak generálása a leírás alapján.
- Releváns CTA generálása és külön mezőben tárolása.
- Az eredmények mentése JSON fájlba.
"""

from PIL import Image
import json
import os
from google.colab import drive
from transformers import BlipProcessor, BlipForConditionalGeneration, pipeline
from keybert import KeyBERT
import torch
from datetime import datetime

# Fájlkezelő osztály a Google Drive és JSON műveletekhez
class FileManager:
    def __init__(self, project_dir='/content/drive/MyDrive/Image', filename='cleaned_output.json'):
        """Inicializálja a fájlkezelőt a megadott könyvtár- és fájlnévvel."""
        self.project_dir = project_dir  # Projekt könyvtár elérési útja
        self.filename = os.path.join(project_dir, filename)  # JSON fájl teljes elérési útja
        self.data = {}  # Adatok tárolása

    def mount_drive(self):
        """Csatlakoztatja a Google Drive-ot a Colab környezetben."""
        print("Mounting Google Drive...")
        drive.mount('/content/drive', force_remount=True)

    def load_json(self):
        """Betölti a meglévő JSON fájlt, vagy létrehoz egy újat, ha nem létezik."""
        if os.path.exists(self.filename):
            try:
                with open(self.filename, 'r', encoding='utf-8') as f:
                    self.data = json.load(f)
            except json.JSONDecodeError:
                print(f"Figyelmeztetés: '{self.filename}' érvénytelen JSON fájl.")
                self.data = {}
        else:
            print(f"'{self.filename}' nem található. Új fájl lesz létrehozva.")
        return self.data

    def save_json(self):
        """Elmenti az adatokat a JSON fájlba."""
        try:
            with open(self.filename, 'w', encoding='utf-8') as f:
                json.dump(self.data, f, ensure_ascii=False, indent=2)
            print(f"'{self.filename}' mentve!")
        except Exception as e:
            print(f"Hiba a fájl mentése közben: {e}")

    def list_images(self):
        """Kilistázza a megadott könyvtárban lévő képeket (.png, .jpg, .jpeg kiterjesztéssel)."""
        if not os.path.exists(self.project_dir):
            print(f"Hiba: '{self.project_dir}' könyvtár nem található.")
            return []
        return [f for f in os.listdir(self.project_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Kép- és SEO-elemző osztály
class ImageDescriber:
    def __init__(self, processor, model):
        """Inicializálja a képelemzőt a BLIP modellel és SEO eszközökkel."""
        self.processor = processor  # BLIP processzor
        self.model = model  # BLIP modell
        # SEO és kulcsszó generáláshoz
        self.summary_generator = pipeline("summarization", model="facebook/bart-large-cnn", framework="pt")
        self.kw_model = KeyBERT()  # Kulcsszókinyerő modell

    def load_image(self, image_path):
        """Betölti a képet RGB formátumba."""
        try:
            return Image.open(image_path).convert("RGB")
        except Exception as e:
            print(f"Hiba a(z) '{image_path}' betöltése közben: {e}")
            return None

    def generate_description(self, image_path):
        """Generál egy ~500 karakteres leírást a képről a BLIP modellel."""
        image = self.load_image(image_path)
        if image is None:
            return "Nem sikerült betölteni a képet."
        inputs = self.processor(image, return_tensors="pt").to('cuda')
        out = self.model.generate(
            **inputs,
            max_new_tokens=300,  # Célzottan ~500 karakter
            num_beams=4,
            length_penalty=2.7,  # Erősebb ösztönzés hosszabb szövegre
            no_repeat_ngram_size=3,  # Zagyvaság csökkentése
            do_sample=False,
            early_stopping=False
        )
        desc = self.processor.decode(out[0], skip_special_tokens=True)
        if len(desc) > 500:
            desc = desc[:500]  # Maximum 500 karakter
        return desc

    def generate_relevant_cta(self, image_description):
        """Generál egy releváns CTA-t a leírás kulcsszavai alapján, félkövérrel és emojikkal."""
        desc_lower = image_description.lower()
        cta_options = {
            "woman": "Get inspired by this style!",
            "man": "Check out this look!",
            "dog": "Meet this adorable pet!",
            "cat": "Meet this cute cat!",
            "street": "Explore this urban vibe!",
            "sun": "Enjoy this sunny moment!",
            "leaves": "Admire this autumn scene!",
            "salon": "Visit this pet salon!"
        }
        for keyword, cta_text in cta_options.items():
            if keyword in desc_lower:
                return f"📌 **{cta_text}** ✨"
        return f"📌 **Discover this scene!** ✨"  # Alapértelmezett

    def generate_seo_elements(self, image_description):
        """Generál SEO címet, leírást és kulcsszavakat az image_description alapján."""
        # SEO cím (max 100 karakter)
        summary_input = f"Summarize for SEO title: {image_description}"
        summary = self.summary_generator(summary_input, max_length=25, min_length=5, do_sample=False)[0]['summary_text']
        seo_title = f"{summary[:80].strip()} - Pin It!"
        if len(seo_title) > 100:
            seo_title = seo_title[:97] + "..."

        # SEO leírás (max 160 karakter)
        desc_input = f"Summarize for SEO description: {image_description}"
        seo_desc = self.summary_generator(desc_input, max_length=90, min_length=20, do_sample=False)[0]['summary_text']
        if len(seo_desc) > 160:
            seo_desc = seo_desc[:157] + "..."
        if "kitchen" in seo_desc.lower() and "salon" in image_description.lower():
            seo_desc = seo_desc.replace("kitchen", "salon")

        # 5 releváns kulcsszó a seo_description-ból
        keywords = [kw[0] for kw in self.kw_model.extract_keywords(seo_desc, keyphrase_ngram_range=(1, 2), stop_words='english', top_n=12)[:5]]

        return {
            "seo_title": seo_title,
            "seo_description": seo_desc,
            "keywords": keywords
        }

def main():
    """Fő függvény, amely a teljes feldolgozási folyamatot vezérli."""
    # GPU elérhetőség ellenőrzése
    if not torch.cuda.is_available():
        print("Figyelmeztetés: GPU nem elérhető. A feldolgozás lassabb lehet.")
    else:
        print("GPU észlelve. GPU-t használjuk a feldolgozáshoz.")

    # Fájlkezelő inicializálása és Drive csatlakoztatása
    file_manager = FileManager()
    file_manager.mount_drive()
    data = file_manager.load_json()
    image_files = file_manager.list_images()

    # Képfeldolgozás ellenőrzése
    if not image_files:
        print("Nincsenek képek a megadott könyvtárban.")
        return

    print(f"Talált képek: {image_files}")

    # Modell inicializálása
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to('cuda')
    describer = ImageDescriber(processor, model)

    # Képfeldolgozás és SEO elemzés
    for filename in image_files:
        image_path = os.path.join(file_manager.project_dir, filename)
        print(f"Feldolgozás: {filename}...")

        try:
            prompt = data.get(filename, {}).get("prompt", None)  # Csak tárolásra
            image_desc = describer.generate_description(image_path)
            cta = describer.generate_relevant_cta(image_desc)
            total_length = len(image_desc) + len(cta)
            print(f"{filename} leírása: {image_desc} (Hossz: {len(image_desc)})")
            print(f"{filename} CTA: {cta} (Hossz: {len(cta)}, Összesen: {total_length})")

            # SEO elemek generálása
            seo_elements = describer.generate_seo_elements(image_desc)

            processed_at = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            if filename not in data:
                data[filename] = {}
            data[filename].update({
                "prompt": prompt or "Nincs megadva prompt",
                "image_description": image_desc,
                "seo_title": seo_elements["seo_title"],
                "seo_description": seo_elements["seo_description"],
                "keywords": seo_elements["keywords"],
                "cta": cta,
                "link": "",
                "processed_at": processed_at,
                "uploaded_at": ""
            })
        except Exception as e:
            print(f"Hiba a(z) '{filename}' feldolgozása közben: {e}")

    # Eredmények mentése
    file_manager.data = data
    file_manager.save_json()

if __name__ == "__main__":
    main()

### Paraméterezhető SEO cím és leírás kulcsszavakkal

In [6]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
Képfeldolgozó és SEO-elemző szkript
Ez a szkript a Google Colab környezetben fut, és a következőket végzi:
- Képek leírásának generálása BLIP modellel (~500 karakter).
- SEO-barát cím, leírás és kulcsszavak generálása a leírás alapján.
- Releváns CTA generálása és külön mezőben tárolása.
- Az eredmények mentése JSON fájlba.

A finomhangolható paraméterek modellenként külön szótárakban vannak definiálva a script elején,
hogy könnyen módosíthatóak legyenek a kimenet testreszabásához.
"""

from PIL import Image
import json
import os
from google.colab import drive
from transformers import BlipProcessor, BlipForConditionalGeneration, pipeline
from keybert import KeyBERT
import torch
from datetime import datetime

# Paraméterek konfigurációja
# BLIP modell paraméterei (képleírás generáláshoz)
BLIP_CONFIG = {
    "max_new_tokens": 300,  # Maximális új tokenek száma a generáláshoz (~500 karakter)
    # Magyarázat: Meghatározza a generálandó új tokenek maximális számát. Egy token ~1.5-2 karakter,
    # így ez a leírás hosszát szabályozza. Növelés (pl. 320) hosszabb, csökkentés (pl. 280) rövidebb leírást eredményez.
    "num_beams": 4,  # Beam search száma a legjobb szekvenciák kiválasztásához
    # Magyarázat: Minél nagyobb, annál jobb koherencia, de lassabb. 3 stabil, 5 részletgazdag, de terhelő lehet.
    "length_penalty": 2.7,  # Hossz büntetési tényező
    # Magyarázat: >1 hosszabb, <1 rövidebb szöveget ösztönöz. 2.5-3.0 hosszabb, 2.0 alatti rövidebb leírást ad.
    "no_repeat_ngram_size": 4,  # Ismétlődő n-gramok száma, amit el kell kerülni
    # Magyarázat: Csökkenti a redundanciát. 2 rugalmas, 4 szigorúbb, 3 kompromisszum.
    "do_sample": False,  # Determinisztikus vs. sztochasztikus generálás
    # Magyarázat: False konzisztens, True kreatívabb, de instabil. True esetén temperature finomhangolható.
    "early_stopping": False  # Korai leállítás a generálásnál
    # Magyarázat: False a max_new_tokens-ig megy, True korán leáll, ha befejezettnek ítéli.
}

# BART modell paraméterei (SEO elemek generálásához)
BART_CONFIG = {
    "title_max_length": 25,  # SEO cím maximális tokenhossza
    # Magyarázat: A generált cím token maximális száma (~100 karakter). 20-30 között finomítható.
    "title_min_length": 5,  # SEO cím minimális tokenhossza
    # Magyarázat: Biztosítja a minimális tartalmat. 5-10 között állítható.
    "desc_max_length": 100,  # SEO leírás maximális tokenhossza
    # Magyarázat: A generált leírás token maximális száma (~160 karakter). 80-120 között finomítható.
    "desc_min_length": 20,  # SEO leírás minimális tokenhossza
    # Magyarázat: Minimális tartalom biztosítása. 15-30 között állítható.
    "do_sample": False  # Determinisztikus vs. sztochasztikus generálás
    # Magyarázat: False konzisztens, True kreatívabb. True esetén temperature (pl. 0.7) finomhangolható.
    # temperature alapértelmezett 1.0, alacsonyabb (0.8) konzervatívabb, magasabb (1.2) kreatívabb.
}

class FileManager:
    def __init__(self, project_dir='/content/drive/MyDrive/Image', filename='cleaned_prompts.json'):
        """Inicializálja a fájlkezelőt a megadott könyvtár- és fájlnévvel."""
        self.project_dir = project_dir
        self.filename = os.path.join(project_dir, filename)
        self.data = {}

    def mount_drive(self):
        """Csatlakoztatja a Google Drive-ot a Colab környezetben."""
        print("Mounting Google Drive...")
        drive.mount('/content/drive', force_remount=True)

    def load_json(self):
        """Betölti a meglévő JSON fájlt, vagy létrehoz egy újat, ha nem létezik."""
        if os.path.exists(self.filename):
            try:
                with open(self.filename, 'r', encoding='utf-8') as f:
                    self.data = json.load(f)
            except json.JSONDecodeError:
                print(f"Figyelmeztetés: '{self.filename}' érvénytelen JSON fájl.")
                self.data = {}
        else:
            print(f"'{self.filename}' nem található. Új fájl lesz létrehozva.")
        return self.data

    def save_json(self):
        """Elmenti az adatokat a JSON fájlba."""
        try:
            with open(self.filename, 'w', encoding='utf-8') as f:
                json.dump(self.data, f, ensure_ascii=False, indent=2)
            print(f"'{self.filename}' mentve!")
        except Exception as e:
            print(f"Hiba a fájl mentése közben: {e}")

    def list_images(self):
        """Kilistázza a megadott könyvtárban lévő képeket (.png, .jpg, .jpeg kiterjesztéssel)."""
        if not os.path.exists(self.project_dir):
            print(f"Hiba: '{self.project_dir}' könyvtár nem található.")
            return []
        return [f for f in os.listdir(self.project_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

class ImageDescriber:
    def __init__(self, processor, model):
        """Inicializálja a képelemzőt a BLIP modellel és SEO eszközökkel."""
        self.processor = processor
        self.model = model
        self.summary_generator = pipeline("summarization", model="facebook/bart-large-cnn", framework="pt")
        self.kw_model = KeyBERT()

    def load_image(self, image_path):
        """Betölti a képet RGB formátumba."""
        try:
            return Image.open(image_path).convert("RGB")
        except Exception as e:
            print(f"Hiba a(z) '{image_path}' betöltése közben: {e}")
            return None

    def generate_description(self, image_path):
        """Generál egy ~500 karakteres leírást a képről a BLIP modellel."""
        image = self.load_image(image_path)
        if image is None:
            return "Nem sikerült betölteni a képet."
        inputs = self.processor(image, return_tensors="pt").to('cuda')
        out = self.model.generate(
            **inputs,
            max_new_tokens=BLIP_CONFIG["max_new_tokens"],
            num_beams=BLIP_CONFIG["num_beams"],
            length_penalty=BLIP_CONFIG["length_penalty"],
            no_repeat_ngram_size=BLIP_CONFIG["no_repeat_ngram_size"],
            do_sample=BLIP_CONFIG["do_sample"],
            early_stopping=BLIP_CONFIG["early_stopping"]
        )
        desc = self.processor.decode(out[0], skip_special_tokens=True)
        if len(desc) > 500:
            desc = desc[:500]
        return desc

    def generate_relevant_cta(self, image_description):
        """Generál egy releváns CTA-t a leírás kulcsszavai alapján, félkövérrel és emojikkal."""
        desc_lower = image_description.lower()
        cta_options = {
            "woman": "Get inspired by this style!",
            "man": "Check out this look!",
            "dog": "Meet this adorable pet!",
            "cat": "Meet this cute cat!",
            "street": "Explore this urban vibe!",
            "sun": "Enjoy this sunny moment!",
            "leaves": "Admire this autumn scene!",
            "salon": "Visit this pet salon!"
        }
        for keyword, cta_text in cta_options.items():
            if keyword in desc_lower:
                return f"📌 **{cta_text}** ✨"
        return f"📌 **Discover this scene!** ✨"

    def generate_seo_elements(self, image_description):
        """Generál SEO címet, leírást és kulcsszavakat az image_description alapján."""
        summary_input = f"Summarize for SEO title: {image_description}"
        summary = self.summary_generator(summary_input,
                                        max_length=BART_CONFIG["title_max_length"],
                                        min_length=BART_CONFIG["title_min_length"],
                                        do_sample=BART_CONFIG["do_sample"])[0]['summary_text']
        seo_title = f"{summary[:80].strip()} - Pin It!"
        if len(seo_title) > 100:
            seo_title = seo_title[:97] + "..."

        desc_input = f"Summarize for SEO description: {image_description}"
        seo_desc = self.summary_generator(desc_input,
                                         max_length=BART_CONFIG["desc_max_length"],
                                         min_length=BART_CONFIG["desc_min_length"],
                                         do_sample=BART_CONFIG["do_sample"])[0]['summary_text']
        if len(seo_desc) > 160:
            seo_desc = seo_desc[:157] + "..."
        if "kitchen" in seo_desc.lower() and "salon" in image_description.lower():
            seo_desc = seo_desc.replace("kitchen", "salon")

        keywords = [kw[0] for kw in self.kw_model.extract_keywords(seo_desc,
                                                                  keyphrase_ngram_range=(1, 2),
                                                                  stop_words='english',
                                                                  top_n=15)[:5]]

        return {
            "seo_title": seo_title,
            "seo_description": seo_desc,
            "keywords": keywords
        }

def main():
    """Fő függvény, amely a teljes feldolgozási folyamatot vezérli."""
    if not torch.cuda.is_available():
        print("Figyelmeztetés: GPU nem elérhető. A feldolgozás lassabb lehet.")
    else:
        print("GPU észlelve. GPU-t használjuk a feldolgozáshoz.")

    file_manager = FileManager()
    file_manager.mount_drive()
    data = file_manager.load_json()
    image_files = file_manager.list_images()

    if not image_files:
        print("Nincsenek képek a megadott könyvtárban.")
        return

    print(f"Talált képek: {image_files}")

    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to('cuda')
    describer = ImageDescriber(processor, model)

    for filename in image_files:
        image_path = os.path.join(file_manager.project_dir, filename)
        print(f"Feldolgozás: {filename}...")

        try:
            prompt = data.get(filename, {}).get("prompt", None)
            image_desc = describer.generate_description(image_path)
            cta = describer.generate_relevant_cta(image_desc)
            total_length = len(image_desc) + len(cta)
            print(f"{filename} leírása: {image_desc} (Hossz: {len(image_desc)})")
            print(f"{filename} CTA: {cta} (Hossz: {len(cta)}, Összesen: {total_length})")

            seo_elements = describer.generate_seo_elements(image_desc)

            processed_at = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            if filename not in data:
                data[filename] = {}
            data[filename].update({
                "prompt": prompt or "Nincs megadva prompt",
                "image_description": image_desc,
                "seo_title": seo_elements["seo_title"],
                "seo_description": seo_elements["seo_description"],
                "keywords": seo_elements["keywords"],
                "cta": cta,
                "link": "",
                "processed_at": processed_at,
                "uploaded_at": ""
            })
        except Exception as e:
            print(f"Hiba a(z) '{filename}' feldolgozása közben: {e}")

    file_manager.data = data
    file_manager.save_json()

if __name__ == "__main__":
    main()

GPU észlelve. GPU-t használjuk a feldolgozáshoz.
Mounting Google Drive...
Mounted at /content/drive
Talált képek: ['Pinterest 2025-03-04-101909-GGUF-Flux-deis-beta-seed-797091519311320-0.png', 'Pinterest 2025-03-04-000150-GGUF-Flux-deis-beta-seed-797091519311320-0.png', 'Pinterest 2025-03-03-232739-GGUF-Flux-deis-beta-seed-797091519311320-0.png', 'Pinterest 2025-03-03-232030-GGUF-Flux-deis-beta-seed-797091519311320-0.png', 'Pinterest 2025-03-03-235528-GGUF-Flux-deis-beta-seed-797091519311320-0.png', 'Pinterest 2025-03-04-000857-GGUF-Flux-deis-beta-seed-797091519311320-0.png', 'Pinterest 2025-03-04-103748-GGUF-Flux-deis-beta-seed-740320524205577-0.png', 'Pinterest 2025-03-04-104314-GGUF-Flux-deis-beta-seed-849961598060900-0.png', 'Pinterest 2025-03-03-233837-GGUF-Flux-deis-beta-seed-797091519311320-0.png', 'Pinterest 2025-03-04-102540-GGUF-Flux-deis-beta-seed-740320524205577-0.png', 'Pinterest 2025-03-04-101054-GGUF-Flux-deis-beta-seed-797091519311320-0.png']


Device set to use cuda:0


Feldolgozás: Pinterest 2025-03-04-101909-GGUF-Flux-deis-beta-seed-797091519311320-0.png...
Pinterest 2025-03-04-101909-GGUF-Flux-deis-beta-seed-797091519311320-0.png leírása: there is a cat that is sitting inside of a cardboard box with its mouth open and its mouth wide open and it ' s looking out of the hole in to the side of the box (Hossz: 161)
Pinterest 2025-03-04-101909-GGUF-Flux-deis-beta-seed-797091519311320-0.png CTA: 📌 **Meet this cute cat!** ✨ (Hossz: 27, Összesen: 188)


Your max_length is set to 100, but your input_length is only 46. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)


Feldolgozás: Pinterest 2025-03-04-000150-GGUF-Flux-deis-beta-seed-797091519311320-0.png...
Pinterest 2025-03-04-000150-GGUF-Flux-deis-beta-seed-797091519311320-0.png leírása: a drawing of a girl in a hat with flowers on her head and a flower in her hair (Hossz: 78)
Pinterest 2025-03-04-000150-GGUF-Flux-deis-beta-seed-797091519311320-0.png CTA: 📌 **Discover this scene!** ✨ (Hossz: 28, Összesen: 106)


...

Feldolgozás: Pinterest 2025-03-04-104314-GGUF-Flux-deis-beta-seed-849961598060900-0.png...
Pinterest 2025-03-04-104314-GGUF-Flux-deis-beta-seed-849961598060900-0.png leírása: there is a dog that is sitting on a table with a stuffed animal in it ' s mouth (Hossz: 79)
Pinterest 2025-03-04-104314-GGUF-Flux-deis-beta-seed-849961598060900-0.png CTA: 📌 **Meet this adorable pet!** ✨ (Hossz: 31, Összesen: 110)


Your max_length is set to 100, but your input_length is only 28. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)


Feldolgozás: Pinterest 2025-03-03-233837-GGUF-Flux-deis-beta-seed-797091519311320-0.png...
Pinterest 2025-03-03-233837-GGUF-Flux-deis-beta-seed-797091519311320-0.png leírása: a black and white drawing of a baby in a hoodedie with his hands in his mouth (Hossz: 77)
Pinterest 2025-03-03-233837-GGUF-Flux-deis-beta-seed-797091519311320-0.png CTA: 📌 **Discover this scene!** ✨ (Hossz: 28, Összesen: 105)


Your max_length is set to 100, but your input_length is only 28. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)


Feldolgozás: Pinterest 2025-03-04-102540-GGUF-Flux-deis-beta-seed-740320524205577-0.png...
Pinterest 2025-03-04-102540-GGUF-Flux-deis-beta-seed-740320524205577-0.png leírása: there is a cat that is sitting in a cardboard box with its mouth open and it ' s mouth wide open, with its mouth wide open and eyes wide open, and mouth wide wide open, looking at the cat is looking at the viewer to the camera, with its tongue wide, with its head, and eyes wide, and its mouth wide, and mouth open, with mouth wide, it ' s eyes wide, with mouth open, and eyes open, and it is looking to the mouth wide, with it ' s wide, and eyes, with its eyes wide, its mouth open, its mouth, and m (Hossz: 500)
Pinterest 2025-03-04-102540-GGUF-Flux-deis-beta-seed-740320524205577-0.png CTA: 📌 **Meet this cute cat!** ✨ (Hossz: 27, Összesen: 527)
Feldolgozás: Pinterest 2025-03-04-101054-GGUF-Flux-deis-beta-seed-797091519311320-0.png...
Pinterest 2025-03-04-101054-GGUF-Flux-deis-beta-seed-797091519311320-0.png leírása: t